In [ ]:
% reload_ext autoreload
% autoreload 2
% matplotlib inline
from IPython import display

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import utils
from lib import pipeline

C = utils.get_config('./myconfig.ini')

# Artstyle

The collection of WikiArt paintings cropped to `224x224` at the centre. See `MNIST.ipynb` for introduction.

In [ ]:
traindata = pipeline.cropped.Data(root=C.paths.root, info_csv=C.paths.train_csv, train=True)

print('{} Instances. {} Classes'.format(len(traindata), len(traindata.encoder.classes_)))

In [ ]:
fig, (row1, row2) = plt.subplots(nrows=2, ncols=5)
for i, ax in enumerate((*row1, *row2)):
    x, y = traindata[4470+i]  # x.shape == (1, 28, 28), y.shape == (1,)
    ax.imshow(x[0])

In [ ]:
EPOCHS = 1
BATCH_SIZE = 10
LRATE = 1e-3
MOMENTUM = 0.9
USE_CUDA = True

net = pipeline.cropped.AlexNet(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=LRATE, momentum=MOMENTUM)

model = pipeline.cropped.Model(net=net, criterion=criterion, optimizer=optimizer, cuda=USE_CUDA)
model.train(dataset=traindata, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
testdata = pipeline.cropped.Data(root=C.paths.root, info_csv=C.paths.train_csv, train=False)
model.evaluate(testdata)

In [ ]:
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
d = DataLoader(traindata, batch_size=10)
for x, y in tqdm(d):
    assert x.shape == (10, 3, 224, 224)